In [1]:
import pandas as pd
import numpy as np

#warning
import warnings
warnings.filterwarnings('ignore')

#sklearn
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

Data Imports (3 Datasets already mentioned)

In [2]:
df_netflix=pd.read_csv(r"C:/Users/User-PC/Movie Recommendation System - DBSCAN Clustering/Netflix/titles.csv")
df_hbo=pd.read_csv(r"C:/Users/User-PC/Movie Recommendation System - DBSCAN Clustering/HBO/titles.csv")
df_amazon=pd.read_csv(r"C:/Users/User-PC/Movie Recommendation System - DBSCAN Clustering/Amazon/titles.csv")

In [3]:
df=pd.concat([df_netflix,df_hbo,df_amazon],axis=0)
df

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,tm510327,Lily Is Here,MOVIE,Dallas and heroin have one thing in common: Du...,2021,NaN,93,['drama'],['US'],NaN,tt7672388,5.3,20.0,1.406,NaN
9867,tm1079144,Jay Nog: Something from Nothing,MOVIE,Something From Nothing takes you on a stand-up...,2021,NaN,55,['comedy'],['US'],NaN,tt15041600,NaN,NaN,0.600,NaN
9868,tm847725,Chasing,MOVIE,A cop from Chennai sets out to nab a dreaded d...,2021,NaN,116,['crime'],['IN'],NaN,NaN,NaN,NaN,1.960,NaN
9869,tm1054116,Baikunth,MOVIE,"This story is about prevalent caste problem, e...",2021,NaN,72,"['family', 'drama']",[],NaN,tt14331982,8.4,49.0,0.645,NaN


Data Cleaning & Preprocessing

In [4]:
df.duplicated().sum()
df_movies=df.drop_duplicates()

In [5]:
df_movies.duplicated().sum()

0

In [6]:
df_movies.drop(['description','age_certification'],axis=1,inplace=True)

In [7]:
df_movies['lead_prod_countries']=df_movies['production_countries'].str.replace(r"\['",'',regex=True).str.replace(r"\[",'',regex=True).str.replace(r"'",'',regex=True).str.replace(r"\]",'',regex=True).str.split(',').str[0]
df_movies['lead_prod_countries']=df_movies['lead_prod_countries'].replace('',np.nan)

In [8]:
df_movies

,id,title,type,release_year,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,lead_prod_countries
0,ts300399,Five Came Back: The Reference Films,SHOW,1945,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN,US
1,tm84618,Taxi Driver,MOVIE,1976,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,US
2,tm154986,Deliverance,MOVIE,1972,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300,US
3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811,GB
4,tm120801,The Dirty Dozen,MOVIE,1967,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600,GB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,tm510327,Lily Is Here,MOVIE,2021,93,['drama'],['US'],NaN,tt7672388,5.3,20.0,1.406,NaN,US
9867,tm1079144,Jay Nog: Something from Nothing,MOVIE,2021,55,['comedy'],['US'],NaN,tt15041600,NaN,NaN,0.600,NaN,US
9868,tm847725,Chasing,MOVIE,2021,116,['crime'],['IN'],NaN,NaN,NaN,NaN,1.960,NaN,IN
9869,tm1054116,Baikunth,MOVIE,2021,72,"['family', 'drama']",[],NaN,tt14331982,8.4,49.0,0.645,NaN,NaN


In [9]:
#Working with genres
df_movies['movie_genre']=df_movies['genres'].str.replace(r"\['",'',regex=True).str.replace(r"']",'',regex=True).str.replace(r"'",'',regex=True).str.split(',').str[0]
df_movies['movie_genre']=df_movies['movie_genre'].replace('',np.nan)

In [10]:
df_movies

,id,title,type,release_year,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,lead_prod_countries,movie_genre
0,ts300399,Five Came Back: The Reference Films,SHOW,1945,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN,US,documentation
1,tm84618,Taxi Driver,MOVIE,1976,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,US,drama
2,tm154986,Deliverance,MOVIE,1972,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300,US,drama
3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811,GB,fantasy
4,tm120801,The Dirty Dozen,MOVIE,1967,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600,GB,war
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,tm510327,Lily Is Here,MOVIE,2021,93,['drama'],['US'],NaN,tt7672388,5.3,20.0,1.406,NaN,US,drama
9867,tm1079144,Jay Nog: Something from Nothing,MOVIE,2021,55,['comedy'],['US'],NaN,tt15041600,NaN,NaN,0.600,NaN,US,comedy
9868,tm847725,Chasing,MOVIE,2021,116,['crime'],['IN'],NaN,NaN,NaN,NaN,1.960,NaN,IN,crime
9869,tm1054116,Baikunth,MOVIE,2021,72,"['family', 'drama']",[],NaN,tt14331982,8.4,49.0,0.645,NaN,NaN,family


In [11]:
df_movies.drop(columns=['genres','production_countries'],axis=1,inplace=True)

Drop Missing Values

In [12]:
df_movies.shape

(18980, 13)

In [13]:
df_movies.isnull().sum()

id                         0
title                      1
type                       0
release_year               0
runtime                    0
seasons                14772
imdb_id                 1394
imdb_score              1873
imdb_votes              1910
tmdb_popularity          670
tmdb_score              2656
lead_prod_countries     1160
movie_genre                0
dtype: int64

In [14]:
df_movies.dropna(inplace=True)

In [15]:
df_movies.set_index('title',inplace=True)
df_movies.drop(['id','imdb_id'],axis=1, inplace=True)

In [16]:
df_movies.isnull().sum()

type                   0
release_year           0
runtime                0
seasons                0
imdb_score             0
imdb_votes             0
tmdb_popularity        0
tmdb_score             0
lead_prod_countries    0
movie_genre            0
dtype: int64

Encoding Categorical Features

In [17]:
dummies=pd.get_dummies(df_movies[['type','lead_prod_countries','movie_genre']],drop_first=True)

df_movies_dum=pd.concat([df_movies,dummies],axis=1)

df_movies_dum.drop(['type','lead_prod_countries','movie_genre'],axis=1, inplace=True)

In [18]:
df_movies_dum

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,lead_prod_countries_AR,lead_prod_countries_AT,lead_prod_countries_AU,...,movie_genre_history,movie_genre_horror,movie_genre_music,movie_genre_reality,movie_genre_romance,movie_genre_scifi,movie_genre_sport,movie_genre_thriller,movie_genre_war,movie_genre_western
title,,,,,,,,,,,,,,,,,,,,,
Monty Python's Flying Circus,1969,30,4.0,8.8,73424.0,17.617,8.306,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Seinfeld,1989,24,9.0,8.9,308824.0,130.213,8.301,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Knight Rider,1982,51,4.0,6.9,34115.0,50.267,7.500,False,False,False,...,False,False,False,False,False,True,False,False,False,False
Thomas & Friends,1984,10,24.0,6.5,5104.0,42.196,6.500,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Saved by the Bell,1989,23,5.0,7.1,35034.0,19.855,8.000,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Putham Pudhu Kaalai: Vidiyaadha,2022,29,1.0,6.2,389.0,1.400,8.000,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chivas: El Rebaño Sagrado,2021,46,1.0,7.5,46.0,18.308,7.500,False,False,False,...,False,False,False,False,False,False,True,False,False,False
Be Yourself,2021,47,1.0,8.1,29.0,6.259,7.000,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Scaling (Min-Max Scaling)

In [19]:
scaler=MinMaxScaler()
df_scaled=scaler.fit_transform(df_movies_dum)
df_scaled=pd.DataFrame(df_scaled,columns=df_movies_dum.columns)

In [20]:
df_scaled

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,lead_prod_countries_AR,lead_prod_countries_AT,lead_prod_countries_AU,...,movie_genre_history,movie_genre_horror,movie_genre_music,movie_genre_reality,movie_genre_romance,movie_genre_scifi,movie_genre_sport,movie_genre_thriller,movie_genre_war,movie_genre_western
0,0.397727,0.168539,0.058824,0.9125,0.037009,0.007913,0.815870,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.625000,0.134831,0.156863,0.9250,0.155671,0.058490,0.815326,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.545455,0.286517,0.058824,0.6750,0.017194,0.022579,0.728261,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.568182,0.056180,0.450980,0.6250,0.002570,0.018954,0.619565,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.625000,0.129213,0.078431,0.7000,0.017658,0.008919,0.782609,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3296,1.000000,0.162921,0.000000,0.5875,0.000194,0.000629,0.782609,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3297,0.988636,0.258427,0.000000,0.7500,0.000021,0.008224,0.728261,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3298,0.988636,0.264045,0.000000,0.8250,0.000012,0.002811,0.673913,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3299,0.988636,0.146067,0.000000,0.2750,0.000490,0.000628,0.271739,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


DBSCAN

In [21]:
eps_array=[0.2,0.5,1]
min_samples_array=[5,10,30]

for eps in eps_array:
    for min_samples in min_samples_array:
        clusters=DBSCAN(eps=eps,min_samples=min_samples)
        clusters.fit(df_scaled)
        cluster_labels=clusters.labels_

        if len(set(cluster_labels))==1:
            continue
        silhoutte_avg=silhouette_score(df_scaled,cluster_labels)

        print("For eps =",eps,
             "For min samples =",min_samples,
             "Count Cluster =",len(set(cluster_labels)),
              "The average silhoutte score is ",silhoutte_avg)

For eps = 0.2 For min samples = 5 Count Cluster = 75 The average silhoutte score is  0.53612056875441
For eps = 0.2 For min samples = 10 Count Cluster = 38 The average silhoutte score is  0.43346464033479776
For eps = 0.2 For min samples = 30 Count Cluster = 17 The average silhoutte score is  0.29149158112321844
For eps = 0.5 For min samples = 5 Count Cluster = 92 The average silhoutte score is  0.6353504043434447
For eps = 0.5 For min samples = 10 Count Cluster = 56 The average silhoutte score is  0.5902309656371962
For eps = 0.5 For min samples = 30 Count Cluster = 21 The average silhoutte score is  0.4080911241370476
For eps = 1 For min samples = 5 Count Cluster = 94 The average silhoutte score is  0.6411052242503918
For eps = 1 For min samples = 10 Count Cluster = 57 The average silhoutte score is  0.595949653979977
For eps = 1 For min samples = 30 Count Cluster = 22 The average silhoutte score is  0.41829037322577756


DBSCAN With Best Hyperparamters (eps=1, minpoints=5)

In [22]:
dbscan_model=DBSCAN(eps=1,min_samples=5).fit(df_scaled)

print("Clusters",len(set(dbscan_model.labels_)))
print("Score",silhouette_score(df_scaled,dbscan_model.labels_))

Clusters 94
Score 0.6411052242503918


Save Clusters for Recommendation

In [23]:
df_movies['dbscan_clusters']=dbscan_model.labels_

In [24]:
df_movies['dbscan_clusters'].value_counts()

dbscan_clusters
-1     389
 1     347
 13    267
 35    221
 5     145
      ... 
 87      5
 88      5
 42      5
 52      5
 56      5
Name: count, Length: 94, dtype: int64

Movie Recommendation Function

In [49]:
import random

def recommend_movie(movie_name: str):
    #movie name to lowrcase for case insensitive matching
    movie_name=movie_name.lower()
    #new column 'name' to store titles values for comparison (lower case)
    df_movies['name']= df_movies.index.str.lower()
    #find the movie which matches the inpute name
    movie=df_movies[df_movies['name'].str.contains(movie_name, na=False)]

    if not movie.empty:
        #Get the cluster label
        clusters=movie['dbscan_clusters'].values[0]
        #get all the movies of the same clusters
        cluster_movies=df_movies[df_movies['dbscan_clusters']==clusters]
        #select only 5 movies in the clusters
        if len(cluster_movies) >= 5:
            #randomly select 5
            recommended_movies =random.sample(list(cluster_movies.index),5)
        else:
            #if fewer then 5, return all the movies in the cluster
            recommended_movies=list(cluster_movies.index)
        #print the recommended movies
        print("We can recommend you these movies:")
        for m in recommended_movies:
            print(m)
    else:
        print("Movie not found in the Database.")

In [50]:
i=input("Input Movie name: ")
print("\n\n")
recommend_movie(i)

Input Movie name:  Vampire Knight





We can recommend you these movies
Eden
DEATH NOTE
Violet Evergarden
Kengan Ashura
Banana Fish


In [51]:
df_movies.to_csv("clustered_movies.csv",index=False)